In [1]:
import numpy as np
import pandas as pd
import scipy.io.wavfile as wavfile
import os
import numpy.random as rng
from scipy import signal
import matplotlib.pyplot as plt
import pickle

def rms(sig):
    return np.sqrt(np.mean(sig**2))
def db(PSD):
    return 10*np.log10(PSD)

In [3]:
def butter_lowpass(cutoff, fs, order=10):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='low', analog=False)
    return b, a
def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

In [5]:
import colorednoise as cn
import librosa

target_rms = 0.2
base_dir = '/Users/girishhanumantha/Documents/GitHub/audio_anomaly_detection/data/external/unprocessed_audio/'
src_list = ['FOXSPORTS_06-09-2022_09-00-00_PM_28518.mp3', 'KDMX-FM_06-09-2022_10-00-00_PM_28508.mp3',
            'KEGL-FM_06-09-2022_10-00-00_PM_28507.mp3']
src_names = ['FOXSPORTS', 'KDMX-FM', 'KEGL-FM']
# src = '/Users/langchenfan/PycharmProjects/AnomalyDetection/unprocessed_audio/Mono/KDMX-FM_06-09-2022_10-00-00_PM_28508.mp3'
# src = '/Users/langchenfan/PycharmProjects/AnomalyDetection/unprocessed_audio/Mono/KEGL-FM_06-09-2022_10-00-00_PM_28507.mp3'
#src = '/Users/langchenfan/Documents/Radio Anomaly Detection/new_download/KTST-FM_09-01-2022_01-00-00_PM_51477.mp3'
#### don't forget to change the output file station!!!
for src in range(len(src_list)):
    wav0, fs = librosa.load(base_dir + src_list[src], sr=44100)
    n = 10 * fs  #audio clip len
    for j in range(540):
        if j % 10 < 2:
            continue
        else:
            wav = wav0[j * n:(j + 1) * n]
            # wav = wav/rms(wav)*target_rms
            if j % 10 == 2 or j % 10 == 4:
                # generate noise piece at random place
                # noise_len = fs*round(rng.default_rng().uniform(low=3,high=8)) #noise length from 10 to 15 sec
                # noise_start = rng.default_rng().integers(2*fs)
                # noise = rng.default_rng().uniform(size=noise_len)
                # noise = cn.powerlaw_psd_gaussian(0,noise_len,15000/fs)
                # weight = (np.abs(signal.hilbert(wav[noise_start:noise_start+noise_len]))+0.08)*0.7
                # amp = rng.default_rng().uniform(low=0.05,high=0.3)
                # addon = np.multiply(noise*amp,weight)
                # wav[noise_start:noise_start+noise_len] += addon

                # add noise to entire piece
                noise = cn.powerlaw_psd_gaussian(0, n, 15000 / fs)
                weight = (np.abs(signal.hilbert(wav)) + 0.08) * 0.7
                amp = rng.default_rng().uniform(low=0.05, high=0.3)
                addon = np.multiply(noise * amp, weight)
                wav += addon
                wav = butter_lowpass_filter(wav, 18000, fs, order=30)
                fname = '/Users/girishhanumantha/Documents/GitHub/audio_anomaly_detection/data/external/SimNoise/' + src_names[
                    src] + str(j + 1) + '_whitenoise'
                # wavfile.write('/Users/langchenfan/Documents/Radio Anomaly Detection/SimNoise/noise'+str(j+1)+'.wav',rate=fs,data=addon)
                wavfile.write(fname + '.wav', rate=fs, data=wav)  #write wav files
            elif j % 10 == 6 or j % 10 == 8:
                fname = '/Users/girishhanumantha/Documents/GitHub/audio_anomaly_detection/data/external/CleanSim/' + src_names[
                    src] + str(j + 1) + '_inf'
                wavfile.write(fname + '.wav', rate=fs, data=wav)  #write wav files

/Users/girishhanumantha/Documents/GitHub/audio_anomaly_detection/venv/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/Users/girishhanumantha/Documents/GitHub/audio_anomaly_detection/venv/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
/Users/girishhanumantha/Documents/GitHub/audio_anomaly_detection/venv/lib/python3.7/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
